In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import os

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy.stats as stats

import seaborn as sns

from datetime import datetime

import re

from tqdm.notebook import tqdm

from tools import * 

config = pd.Series({
    'devide_by':5,
    'year':2019,
    'datafrom': ['BicingNou_ESTACIONS', 'BicingNou_INFORMACIO'],
    'dataset': 'BicingNou_MOD',
    'ttl': 30,
    'month': 4,
    'monthname': 'Abril'
})

os.system(f"mkdir -p ../dades/{config.year}/{config.dataset}")


0

In [2]:
%%html
<style>
    p {
        float:left
    }  
    table {
        display: inline-block
    }
</style>

<div>
    <table>
        <tr>
            <th><p>Camp<p><th>
            <th><p>Descripció<p><th>
        <tr>
        <tr>
            <td><p>last_updated<p><td>
            <td><p>Timestamp de l'arxiu<p><td>
        <tr>
        <tr>
            <td><p>ttl<p><td>
            <td><p>TimeToLive de la resposta<p><td>
        <tr>
        <tr>
            <td><p>data<p><td>
            <td><p>Contenidor d'arrays d'informació d'estacions<p><td>
        <tr>
        <tr>
            <td><p>stations<p><td>
            <td><p>Array de dades de cada estació<p><td>
        <tr>
        <tr>
            <td><p>station_id<p><td>
            <td><p>Identificador de l'estació<p><td>
        <tr>
        <tr>
            <td><p>num_bikes_available<p><td>
            <td><p>Nombre de bicicletes disponibles<p><td>
        <tr>
        <tr>
            <td><p>num_bikes_available_types<p><td>
            <td><p>Array de tipus de bicicletes disponibles<p><td>
        <tr>
        <tr>
            <td><p>mechanical<p><td>
            <td><p>Nombre de bicicletes mecàniques disponibles<p><td>
        <tr>
        <tr>
            <td><p>ebike<p><td>
            <td><p>Nombre de bicicletes elèctriques disponibles<p><td>
        <tr>
        <tr>
            <td><p>num_docks_available<p><td>
            <td><p>Nombre de ancoratges disponibles<p><td>
        <tr>
        <tr>
            <td><p>is_installed<p><td>
            <td><p>L'estació està correctament instalada (0-NO,1-SI)<p><td>
        <tr>
        <tr>
            <td><p>is_renting<p><td>
            <td><p>L'estació està proporcionant bicicletes correctament<p><td>
        <tr>
        <tr>
            <td><p>is_returning<p><td>
            <td><p>L'estació està ancorant bicicletes correctament<p><td>
        <tr>
        <tr>
            <td><p>last_reported<p><td>
            <td><p>Timestamp de la informació de l'estació<p><td>
        <tr>
        <tr>
            <td><p>is_charging_station<p><td>
            <td><p>L'estació té capacitat de càrrega de bicicletes elèctriques<p><td>
        <tr>
        <tr>
            <td><p>status<p><td>
            <td><p>Estat de l'estació (IN_SERVICE=En servei, CLOSED=Tancada)<p><td>
        <tr>
    <table>
<div>

# New data 2019 Abril 

## dades_2019_Abril_info

### Preprocessing the data 

In [ ]:
%%time

dades_2019_Abril_info = pd.read_csv(f'../dades/{config.year}/{config.datafrom[0]}/{config.year}_{config.month:02d}_{config.monthname}_{config.datafrom[0]}.csv', low_memory=False)

intial_size = dades_2019_Abril_info.shape[0]
print(dades_2019_Abril_info.shape)

# change column to one hot enconding
dades_2019_Abril_info['is_charging_station'] = dades_2019_Abril_info.is_charging_station.astype(np.int)

# STATUS = IN_SERVICE=En servei, CLOSED=Tancada
# replace IN_SERVICE with 1 and CLOSED with 0
dades_2019_Abril_info['status'].replace(to_replace=['NOT_IN_SERVICE', 'IN_SERVICE'], value=[0, 1], inplace=True)

dades_2019_Abril_info.loc[dades_2019_Abril_info.last_reported.isna(), 'last_reported'] = dades_2019_Abril_info.loc[dades_2019_Abril_info.last_reported.isna(), 'last_updated']

# will remove the duplicate for last reported for all stations in the dataset
dades_2019_Abril_info = remove_duplicates_all(dades_2019_Abril_info.copy(), 'last_reported')

# convert timestamps of last_updated
dades_2019_Abril_info = convert_timestamp(dades_2019_Abril_info.copy(), ['last_updated'], sort=True, add=True)

# convert timestamps to multimple of 3
dades_2019_Abril_info = timestamp_multipleof(
    devide_by=config.devide_by, 
    column='minutes_last_updated_date',
    df=dades_2019_Abril_info.copy(), 
    new_column='last_updated', 
    year_column='year_last_updated_date',
    month_column='month_last_updated_date',
    day_column='dayofmonth_last_updated_date',
    hour_column='hour_last_updated_date',
    minutes_column='minutes_last_updated_date'
)    

# drop not needed columns
dades_2019_Abril_info.drop(
    [
        'year_last_updated_date', 'month_last_updated_date',
        'week_last_updated_date', 'dayofweek_last_updated_date',
        'dayofmonth_last_updated_date', 'dayofyear_last_updated_date',
        'hour_last_updated_date', 'minutes_last_updated_date'
    ], 
    axis=1, 
    inplace=True
)

print(dades_2019_Abril_info.shape)
print('removed:', intial_size-dades_2019_Abril_info.shape[0])

## dades_2019_Abril_us

### Preprocessing the data 

In [ ]:
%%time

dades_2019_Abril_us = pd.read_csv(f'../dades/{config.year}/{config.datafrom[1]}/{config.year}_{config.month:02d}_{config.monthname}_{config.datafrom[1]}.csv', low_memory=False)

intial_size = dades_2019_Abril_us.shape[0]
print(dades_2019_Abril_us.shape)

dades_2019_Abril_us.loc[dades_2019_Abril_us.altitude.isin(['0.1', 'nan', np.nan]), 'altitude'] = '0'

cond = (~dades_2019_Abril_us.altitude.isin([str(x) for x in range(200)] + [np.nan]))
print(dades_2019_Abril_us[cond].shape)
# 485 row does not have 0 in the altitud column
# capacity is filled with values 1 to fix this we need to shift the data 

# Fix data 
dades_2019_Abril_us.loc[cond, ['capacity']] = dades_2019_Abril_us[cond].post_code
dades_2019_Abril_us.loc[cond, ['post_code']] = dades_2019_Abril_us[cond].address
dades_2019_Abril_us.loc[cond, ['address']] = dades_2019_Abril_us[cond].altitude
dades_2019_Abril_us.loc[cond, ['altitude']] = '0'
dades_2019_Abril_us.altitude.fillna('0', inplace=True)

# will remove the duplicate for last reported for all stations in the dataset
dades_2019_Abril_us = remove_duplicates_all(dades_2019_Abril_us.copy(), 'last_updated')

# post code is wrong need fixing using long & lat. 
# can be fixed using post code data from old dataset after the merge
dades_2019_Abril_us['post_code'] = '0'

dades_2019_Abril_us = convert_timestamp(dades_2019_Abril_us.copy(), ['last_updated'], sort=True, add=True)

# convert timestamps to multimple of 3
dades_2019_Abril_us = timestamp_multipleof(
    devide_by=config.devide_by, 
    column='minutes_last_updated_date',
    df=dades_2019_Abril_us.copy(), 
    new_column='last_updated', 
    year_column='year_last_updated_date',
    month_column='month_last_updated_date',
    day_column='dayofmonth_last_updated_date',
    hour_column='hour_last_updated_date',
    minutes_column='minutes_last_updated_date'
)

# drop not needed columns
dades_2019_Abril_us.drop(
    [
        'year_last_updated_date', 'month_last_updated_date',
        'week_last_updated_date', 'dayofweek_last_updated_date',
        'dayofmonth_last_updated_date', 'dayofyear_last_updated_date',
        'hour_last_updated_date', 'minutes_last_updated_date'
    ],
    axis=1,
    inplace=True
)

dades_2019_Abril_us['physical_configuration'].replace(to_replace=['BIKESTATION', 'ELECTRICBIKESTATION'], value=[0, 1], inplace=True)

# create mew column of last reported and last updated 
dades_2019_Abril_us['street_name'] = dades_2019_Abril_us.apply(
    lambda x: " ".join(re.findall("[a-zA-Z]+", x['name'])),
    axis=1
)

def lambda_fun(name):
    ret = 'nan'
    try:
        ret = re.findall("\d+$", name)[0]
    except:
        ret = 'nan'
        
    return ret

# create mew column of last reported and last updated 
dades_2019_Abril_us['street_number'] = dades_2019_Abril_us.apply(
    lambda x: lambda_fun(x['name']),
    axis=1
)

# we don't have this column anywhere in the new dataset so it got removed
dades_2019_Abril_us.drop(['address', 'name'], axis=1, inplace=True)

print(dades_2019_Abril_us.shape)

print('removed:', intial_size-dades_2019_Abril_us.shape[0])

## merge data us with data info

In [ ]:
%%time

merged_data = dades_2019_Abril_info.merge(
    dades_2019_Abril_us.copy(), 
    left_on=[
        'station_id',
        'last_updated'
    ], 
    right_on=[
        'station_id',
        'last_updated'
    ],
    how='left',
    suffixes=("_stat", "_us")
).copy()

merged_data.shape

In [ ]:
# confirm data is correct
merged_data[merged_data.last_reported.isna()].shape[0], merged_data.shape[0],(merged_data[merged_data.last_reported.isna()].shape[0]/merged_data.shape[0])*100

In [ ]:
merged_data.status.unique()

In [ ]:
%%time

# we don't have this column anywhere in the new dataset so it got removed
merged_data.drop(
    ['ttl_us', 'ttl_stat', 'last_reported'],
    axis=1, 
    inplace=True
)

merged_data[[
    'num_bikes_available',
    'num_bikes_available_types.mechanical', 
    'num_bikes_available_types.ebike', 
    'num_docks_available', 
    'is_installed', 
    'is_renting', 
    'is_returning', 
    'is_charging_station', 
    'status',
    'last_updated'
]] = merged_data[[
    'num_bikes_available',
    'num_bikes_available_types.mechanical', 
    'num_bikes_available_types.ebike', 
    'num_docks_available', 
    'is_installed', 
    'is_renting', 
    'is_returning', 
    'is_charging_station', 
    'status', 
    'last_updated'
]].astype(np.int)

merged_data.altitude = merged_data.altitude.fillna('0').astype(np.int)

merged_data.physical_configuration.fillna(1., inplace=True)

merged_data = correct_columns(merged_data, 'station_id', 'capacity')

merged_data = correct_columns(merged_data, 'station_id', 'lat', take='first')

merged_data = correct_columns(merged_data, 'station_id', 'lon', take='first')

merged_data = correct_columns(merged_data, 'station_id', 'street_name', take='first')

merged_data = correct_columns(merged_data, 'station_id', 'street_number', take='first')

merged_data['post_code'] = '0'

merged_data.post_code.fillna('0', inplace=True)

In [ ]:
print_duplicates(merged_data, ['station_id', 'last_updated'])

In [ ]:
get_features_nans(merged_data)

In [ ]:
get_features_zero(merged_data)

# Cleaning and store of new file

In [ ]:
%%time

merged_data['ttl'] = config.ttl

intial_size = merged_data.shape[0]
print(merged_data.shape)

### will remove the duplicate for last reported for all stations in the dataset
merged_data = remove_duplicates_all(merged_data.copy(), 'last_updated')
# (4305871, 19)

print(merged_data.shape)

print('removed:', intial_size-merged_data.shape[0])

In [ ]:
%%time

merged_data.reset_index(drop=True, inplace=True)

merged_data.drop(['ttl'], axis=1, inplace=True)

# save checkpoint

merged_data.to_csv(f'../dades/{config.year}/{config.dataset}/{config.year}_{config.month:02d}_{config.monthname}_{config.dataset}.csv', index=False)

# re read file
merged_data = pd.read_csv(f'../dades/{config.year}/{config.dataset}/{config.year}_{config.month:02d}_{config.monthname}_{config.dataset}.csv', low_memory=False)

In [ ]:
merged_data.street_name.unique()

In [ ]:
print_duplicates(merged_data.copy(), ['station_id', 'last_updated'])

In [ ]:
get_features_nans(merged_data)

In [ ]:
get_features_zero(merged_data)

In [ ]:
merged_data

# Visualize data

In [ ]:

# re read file
merged_data = pd.read_csv(f'../dades/{config.year}/{config.dataset}/{config.year}_{config.month:02d}_{config.monthname}_{config.dataset}.csv', low_memory=False)

## Study on TTL for station 1
Taking one station ID and looking for the best TTL for multiple answers from the same time


In [ ]:
merged_data_s1 = merged_data[merged_data.station_id == 1]

In [ ]:
merged_data_s1.shape

In [ ]:
get_features_nans(merged_data_s1)

In [ ]:
get_features_zero(merged_data_s1)

In [ ]:
merged_data_s1.street_name.unique()

In [ ]:
merged_data_s1.post_code.unique()

In [ ]:
merged_data_s1.head(5)

### last_updated 

|  | info |
| -- | -- |
|  | last updates is the timestamp for the file |
|  | a station id will never have repeated last_updated timestamp in the dataset |

In [ ]:
(merged_data_s1.last_updated.value_counts() > 1).any()  # give insight if there any last_updated timestamps is repeated 

In [ ]:
show_column_counts(merged_data_s1, 'last_updated')

## study finished

In [ ]:
merged_data.columns

## station_id

In [ ]:
print(stats.describe(merged_data.station_id))
merged_data.station_id.describe()

In [ ]:
# station_id 
# Most values are repeated equally in the dataset
show_column_counts(merged_data, 'station_id')

## num_bikes_available

In [ ]:
# num_bikes_available
# by looking at the differance between the 75% and max number 
# existance of outliers is possible. 54 is too big for a station size
# TODO, check max size of a station? Replace outliers with the maximum size
print(stats.describe(merged_data.num_bikes_available))
merged_data.num_bikes_available.describe()

In [ ]:
show_column_counts(merged_data, 'num_bikes_available')

## num_docks_available

In [ ]:
print(stats.describe(merged_data.num_docks_available))
merged_data.num_docks_available.describe()

In [ ]:
show_column_counts(merged_data, 'num_docks_available')

## capacity

In [ ]:
print(stats.describe(merged_data.capacity))
merged_data.capacity.describe()

In [ ]:
show_column_counts(merged_data, 'capacity')

## num_bikes_available_types.mechanical

In [ ]:
print(stats.describe(merged_data['num_bikes_available_types.mechanical']))
merged_data['num_bikes_available_types.mechanical'].describe()

In [ ]:
show_column_counts(merged_data, 'num_bikes_available_types.mechanical')

## num_bikes_available_types.ebike

In [ ]:
print(stats.describe(merged_data['num_bikes_available_types.ebike']))
merged_data['num_bikes_available_types.ebike'].describe()

In [ ]:
show_column_counts(merged_data, 'num_bikes_available_types.ebike')

## is_installed

In [ ]:
print(stats.describe(merged_data.is_installed))
merged_data.is_installed.describe()

In [ ]:
show_column_counts(merged_data, 'is_installed')

## is_renting

In [ ]:
print(stats.describe(merged_data.is_renting))
merged_data.is_renting.describe()

In [ ]:
show_column_counts(merged_data, 'is_renting')

## is_returning

In [ ]:
print(stats.describe(merged_data.is_returning))
merged_data.is_returning.describe()

In [ ]:
show_column_counts(merged_data, 'is_returning')

## is_charging_station

In [ ]:
print(stats.describe(merged_data.is_charging_station))
merged_data.is_charging_station.describe()

In [ ]:
show_column_counts(merged_data, 'is_charging_station')

## status

In [ ]:
print(stats.describe(merged_data.status))
merged_data.status.describe()

In [ ]:
show_column_counts(merged_data, 'status')

## last_updated

In [ ]:
print(stats.describe(merged_data.last_updated))
merged_data.last_updated.describe()

In [ ]:
show_column_counts(merged_data, 'last_updated')

## physical_configuration

In [ ]:
print(stats.describe(merged_data.physical_configuration))
merged_data.physical_configuration.describe()

In [ ]:
show_column_counts(merged_data, 'physical_configuration')

## altitude

In [ ]:
print(stats.describe(merged_data.altitude))
merged_data.altitude.describe()

In [ ]:
show_column_counts(merged_data, 'altitude')

## post_code

In [ ]:
print(stats.describe(merged_data.post_code))
merged_data.post_code.describe()

In [ ]:
show_column_counts(merged_data, 'post_code')

## lat

In [ ]:
print(stats.describe(merged_data.lat))
merged_data.lat.describe()

In [ ]:
show_column_counts(merged_data, 'lat')

## lon

In [ ]:
print(stats.describe(merged_data.lon))
merged_data.lon.describe()

In [ ]:
show_column_counts(merged_data, 'lon')

## street_name

In [ ]:
print(stats.describe(merged_data.street_name))
merged_data.street_name.describe()

In [ ]:
show_column_counts(merged_data, 'street_name')

## street_number

In [ ]:
print(stats.describe(merged_data.street_number))
merged_data.street_number.describe()

In [ ]:
show_column_counts(merged_data, 'street_number')